In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26466677
paper_name = 'stauffer_powers_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/Table1.txt', sep='\n', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 341 x 1


In [7]:
original_data.head()

,Vacuolar\tProteins
0,PHO91
1,VTC4
2,VCX1
3,CLA4
4,AVT7


In [8]:
original_data['genes'] = original_data['Vacuolar\tProteins'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [10]:
original_data.loc[original_data['genes']=='RPL19-A','genes'] = 'RPL19A'

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                     Vacuolar\tProteins  \
index_input                                               
5                                              V-ATPase   
20           Vacuolar\tProtein\tSorting\t&\tTrafficking   
25                                        AP-3\tcomplex   
30                           CORVET\ttethering\tComplex   
33                                 PI(3,5)P2\tSynthesis   
37                                   ER/Golgi\tProteins   
38                             N\tlinked\tglycosylation   
44                                                   ER   
52                                                Golgi   
61                              Mitochondrial\tProteins   
91                                 Peroxisome\tFunction   
96                                      Cell\tSignaling   
97                                       TOR\tSignaling   
103                                             other\t   
116                    Vesicular\tTransport/endocytosis 

In [13]:
original_data = original_data.loc[t,:]

In [14]:
original_data['data'] = -1

In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['data']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(313, 1)

# Load & process tested strains

In [19]:
# tested = pd.read_excel('raw_data/Deletion Collection List for Anastasia.xlsx', sheet_name='Sheet1', skiprows=2)

In [20]:
# tested.head()

In [21]:
# tested['orf'] = tested['ORF'].astype(str)

In [22]:
# tested['orf'] = clean_orf(tested['orf'])

In [23]:
# tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [24]:
# tested['orf'] = translate_sc(tested['orf'], to='orf')

In [25]:
# # Make sure everything translated ok
# t = looks_like_orf(tested['orf'])
# print(tested.loc[~t,])

In [26]:
# tested.loc[~t,'orf'].values

In [27]:
# tested = tested.loc[t,:]

In [28]:
# tested_orfs = np.unique(tested['orf'].values)

In [29]:
# missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
# missing

In [30]:
# original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [31]:
data = original_data.copy()

In [32]:
dataset_ids = [766]
datasets = datasets.reindex(index=dataset_ids)

In [33]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [34]:
data.head()

dataset_id,766
data_type,value
orf,
YAL002W,-1
YAL021C,-1
YAL056W,-1
YAL061W,-1
YAR030C,-1


## Subset to the genes currently in SGD

In [35]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [36]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,766
,data_type,value
gene_id,orf,
2,YAL002W,-1
19,YAL021C,-1
52,YAL056W,-1
56,YAL061W,-1
1728,YAR030C,-1


# Normalize

In [37]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [38]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [39]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        766        
data_type       value  valuez
gene_id orf                  
2       YAL002W    -1 -3.8318
19      YAL021C    -1 -3.8318
52      YAL056W    -1 -3.8318
56      YAL061W    -1 -3.8318
1728    YAR030C    -1 -3.8318

# Print out

In [40]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [44]:
# from IO.save_data_to_db3 import *

In [45]:
# save_data_to_db(data_all, paper_pmid)